## 데이터 로더(Data Loaders)

이 노트북은 `pyTigerGraph` 에서 **vertex loader** 의 사용을 보여줍니다. 데이터 로더의 역할은 `TigerGraph` 데이터베이스에서 데이터를 가져오는 것입니다. 현재 다음 데이터 로더가 제공됩니다.

* EdgeLoader, 간선의 배치(batch)를 반환합니다.
* VertexLoader, 정점의 배치(batch)를 반환합니다.
* GraphLoader, 무작위로 샘플링된(연결이 끊긴) 하위 그래프(subgraphs)를 pandas의 `dataframe`, `PyG` 또는 `DGL` 형식으로 반환 합니다.
* NeighborLoader, 이웃 샘플링을 사용하여 하위 그래프(subgraphs)를 `dataframe`, `PyG` 또는 `DGL`  형식으로 반환 합니다.
* EdgeNeighborLoader, 간선들에서 이웃 샘플링을 사용하여 하위 그래프(subgraphs)를 `dataframe`, `PyG` 또는 `DGL` 형식으로 반환 합니다.

위의 모든 데이터 로더는 모든 배치를 HTTP 응답(기본값)으로 가져오거나 Kafka를 통해 모든 배치를 스트리밍할 수 있습니다. 전자의 메커니즘은 작은 그래프로 테스트하기에 좋고 빠르지만 데이터 크기 제한이 2GB입니다. 큰 그래프의 경우 크기 제한 및 네트워크 연결 문제로 인해 HTTP 채널이 실패할 수 있습니다. Kafka를 통한 스트리밍은 데이터 견고성과 확장성을 위해 제공됩니다. 또한 Kafka는 다중 소비자(multi-consumer) 사용 사례에 탁월하며 동일한 데이터의 다중 소비자가 있는 경우 모델 검색 또는 하이퍼파라미터 튜닝에 효율적입니다.

데이터 로더는 동종 및 이종 그래프를 모두 지원합니다. 기본적으로 모든 정점 및 간선 유형에서 로드하고 그래프를 동종 그래프로 처리합니다. 그러나 사용자가 로드할 정점 및 간선 유형과 각 유형에서 로드할 속성을 지정할 수도 있습니다. 이 방법으로 사용자는 이기종 그래프 출력을 얻을 수 있습니다.

**NOTE**: 현재 ML Workbench에서 제공하는 모든 기능을 사용하려면 데이터베이스를 한 번만 활성화(activage)해야 합니다. ML Workbench on Cloud를 사용하는 경우 activator가 포함되며 아래 셀을 실행(먼저 주석 해제)하여 활성화(activage)할 수 있습니다. 다른 버전의 Workbench의 경우 https://act.tigergraphlabs.com 에서 활성기(activator)를 다운로드할 수 있습니다. 자세한 지침은 해당 웹사이트에도 포함되어 있습니다.


In [ ]:
# 아래 주석을 제거하고 필요한 정보를 입력하십시오. 자세한 지침은 https://act.tigergraphlabs.com을 참조하십시오.
# !mlwb activate [database address] -u [username] -p [password] -s [secret]

### 데이터베이스 연결 

이 `TigerGraphConnection` 클래스는 TigerGraph 데이터베이스에 대한 연결을 나타냅니다. 후드 아래에는 데이터베이스와 통신하는 데 필요한 정보가 저장됩니다. 꽤 많은 데이터베이스 작업을 수행할 수 있습니다. 자세한 내용은 해당 [문서](https://docs.tigergraph.com/pytigergraph/current/intro/) 를 참조하십시오.

**Note**: 2022년 7월 5일 이후에 생성된 TG 클라우드 DB는 사용자 username/password 대신 Secret가 필요합니다. 그렇지 않으면 비워 둘 수 있습니다.

In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # 데이터베이스 서버의 주소로 변경
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # 2022년 7월 5일 이후에 생성된 TG 클라우드 DB에는 user/pass 대신 secret이 필요합니다.  
)

<span style="color:red">아래 셀의 주석을 제거하고 토큰 인증이 활성화된 경우 토큰을 가져오고 설정하기 위해 실행합니다.</span>. 
* 이것은 tgcloud의 모든 데이터베이스에 필요합니다.
* `<secret>`은 사용자 암호입니다. 자세한 내용은 https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets 를 참조하세요.
* secret을 모르는 경우  `secret=conn.createSecret()`를 사용하여 secret을 만들 수 있습니다.

In [ ]:
#conn.getToken(<secret>)

In [3]:
# 모든 정점 유형에 대한 정점 수
conn.getVertexCount('*')

{'Paper': 2708}

In [4]:
# 모든 간선 유형에 대한 간선 수
conn.getEdgeCount()

{'Cite': 10556}

### 정점 로더(Vertex Loader)

`VertexLoader`는 데이터베이스에서 정점 배치를 가져옵니다. 특히, 정점을 `num_batches`로 나누고 각 배치를 개별적으로 반환합니다. `filter_by`에 제공된 부울 속성은 포함된 꼭짓점을 나타냅니다. 무작위 배치가 필요한 경우 `shuffle`을 True로 설정하십시오.

**Note**: TigerGraph의 그래프에서 로더를 처음 초기화하는 경우 해당 쿼리를 데이터베이스에 설치하고 최적화하므로 초기화에 1분이 걸릴 수 있습니다. 하지만 쿼리 설치는 한 번만 하면 되므로 동일한 TG 그래프에서 다시 로더를 초기화할 때 시간이 걸리지 않습니다.

데이터 로더를 사용하는 방법에는 두 가지가 있습니다. 예시는 [여기](https://github.com/TigerGraph-DevLabs/mlworkbench-docs/blob/main/tutorials/basics/2_dataloaders.ipynb)를 참조하세요.
* 첫째, iterable로 사용할 수 있습니다. 즉, 루프를 통해 모든 데이터 배치를 가져올 수 있습니다. 모든 정점을 한 번에 로드하는 경우(`num_batches=1`), 반복자에는 하나의 배치(모든 정점 중)만 있습니다.
* 둘째, 클래스의 `data` 속성에 직접 접근할 수 있습니다. 로드할 데이터 배치가 하나만 있는 경우 반복자 대신 배치를 직접 제공하므로 이 경우 더 합리적일 수 있습니다. 로드할 데이터 배치가 여러 개인 경우 로더를 다시 반환합니다.

인수(Args):
* attributes (list or dict, optional):
                포함할 정점 속성입니다. 목록인 경우 모든 정점 유형의 목록에 있는 속성이 선택됩니다. 
                특정 속성이 모든 정점 유형에 존재하지 않으면 오류가 발생합니다. 
                dict인 경우 dict의 키는 선택할 정점 유형이고 값은 각 정점 유형에 대해 선택해야 하는 속성의 목록입니다. 
                기본값은 없음입니다.
* batch_size (int, optional):
                각 배치의 정점 수입니다.
                기본값은 없음입니다.
* num_batches (int, optional):
                정점을 분할할 배치 수입니다.
                기본값은 1입니다.
* shuffle (bool, optional):
                데이터를 로드하기 전에 정점을 섞을지 여부입니다.
                기본값은 False입니다.
* filter_by (str, optional):
                포함할 수 있는 꼭짓점을 나타내는 데 사용되는 부울 속성입니다. 기본값은 없음입니다.
* output_format (str, optional):
                로더의 출력 데이터 형식입니다. "데이터 프레임"만 지원됩니다. 기본값은 "데이터 프레임"입니다.
* loader_id (str, optional):
                임의의 문자열이 될 수 있는 로더의 식별자입니다. Kafka 주제 이름으로도 사용됩니다. 
                `None`이면 임의의 문자열이 생성됩니다. 기본값은 없음입니다.
* buffer_size (int, optional):
                메모리에 프리페치하고 저장할 데이터 배치 수입니다. 기본값은 4입니다.
* timeout (int, optional):
                GSQL 쿼리에 대한 시간 초과 값(ms)입니다. 기본값은 300000입니다.

#### 모든 정점을 한 번에 로컬에 직접 로드합니다. 기본(Defalut).

In [6]:
%%time
vertex_loader = conn.gds.vertexLoader(
    num_batches=1, 
    attributes=["id", "x", "y"])
# Note: 각 꼭짓점의 고유 ID(vid)가 자동으로 생성되어 추출됩니다.
# 각 꼭짓점의 기본 ID를 원하면 속성으로 지정합니다.

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 223 ms, sys: 34.9 ms, total: 258 ms
Wall time: 1min 1s


In [7]:
%%time
# `data` 속성을 통해 데이터의 유일한 배치를 가져옵니다.
data = vertex_loader.data
data.shape

CPU times: user 94.4 ms, sys: 33.9 ms, total: 128 ms
Wall time: 225 ms


(2708, 4)

In [8]:
data.head()

,vid,id,x,y
0,0,1596,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,2
1,1,2703,0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...,3
2,2,2379,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...,4
3,3,1480,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,2
4,4,2163,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...,3


#### http를 통해 정점 배치 가져오기

In [9]:
%%time
vertex_loader = conn.gds.vertexLoader( 
    num_batches=20, 
    attributes=["id", "x", "y"],
    shuffle=True,
    filter_by=None
)

CPU times: user 6.27 ms, sys: 1.24 ms, total: 7.52 ms
Wall time: 10.6 ms


In [10]:
%%time
for i, batch in enumerate(vertex_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))

----Batch 0: Shape (131, 4)----
       vid    id                                                  x  y
0  5242900  1395  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  0
----Batch 1: Shape (144, 4)----
       vid    id                                                  x  y
0  6291456  2292  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...  4
----Batch 2: Shape (148, 4)----
       vid    id                                                  x  y
0  7340051  1935  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  3
----Batch 3: Shape (141, 4)----
        vid    id                                                  x  y
0  14680101  1986  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  3
----Batch 4: Shape (149, 4)----
   vid    id                                                  x  y
0   21  1824  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  5
----Batch 5: Shape (150, 4)----
        vid    id                                                  x  y
0  15728645  1459  0 0 0 0 0 0 0

#### 이종(heterogeneous) 그래프의 경우

`Cora`는 동종(homogeneous) 그래프 이므로 다른 그래프에 연결하여 이종(demostrate) 그래프의 사용 사례를 시연합니다.

In [11]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # 데이터베이스 서버의 주소로 변경
    graphname="hetero",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # 2022년 7월 5일 이후에 생성된 TG 클라우드 DB에는 user/pass 대신 secret이 필요합니다. 
)

# 토큰 인증이 활성화된 경우 토큰을 가져오고 설정하려면 아래 주석을 제거하십시오.
#conn.getToken(<secret>) # <secret>은 사용자 암호입니다. 자세한 내용은 https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets를 참조하세요.

In [12]:
print(conn.gsql("ls"))

---- Graph hetero
Vertex Types:
- VERTEX v0(PRIMARY_ID id INT, x LIST<DOUBLE>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v1(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v2(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE v0v0(FROM v0, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v1(FROM v1, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v2(FROM v1, TO v2, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v0(FROM v2, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v1(FROM v2, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v2(FROM v2, TO v2, is_train BOOL, is_val BOOL)

Graphs:
- Graph hetero(v0:v, v1:v, v2:v, v0v0:e, v1v1:e, v1v2:e, v2v0:e,

In [13]:
%%time
loader = conn.gds.vertexLoader(
    attributes={"v0": ["x", "y"],
                "v1": ["x"]},
    batch_size=20,
    shuffle=True,
    filter_by=None,
    loader_id=None,
    buffer_size=4,
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 29.9 ms, sys: 4.49 ms, total: 34.4 ms
Wall time: 39.5 s


In [14]:
%%time
for i, batch in enumerate(loader):
    print("----Batch {}----".format(i))
    for j in batch:
        print("Vertex type:", j)
        print(batch[j].head(1))

----Batch 0----
Vertex type: v0
         vid                                                  x  y
0  163577857  0.386429 0.401847 -0.0926295 -0.0833778 0.6873...  2
Vertex type: v1
         vid                                                  x
0  178257921  -1.02871 -0.135028 -0.138423 -0.89996 1.68272 ...
----Batch 1----
Vertex type: v0
         vid                                                  x  y
0  142606336  -0.417304 -0.464672 -0.229524 0.336801 0.61027...  9
Vertex type: v1
         vid                                                  x
0  170917888  -0.0675672 -0.514478 1.91501 0.96472 -1.40837 ...
----Batch 2----
Vertex type: v0
         vid                                                  x  y
0  139460609  -0.846644 -0.686432 -2.26245 1.24922 0.576238 ...  8
Vertex type: v1
         vid                                                  x
0  197132289  -0.900331 -0.582497 -0.157183 -0.692183 0.7191...
----Batch 3----
Vertex type: v0
         vid                          

### Kafka를 통한 스트리밍 

**Note**: Kafka 스트리밍 기능은 Enterprise Edition에서만 사용할 수 있습니다. 사용하려면 Enterprise Edition을 활성화해야 합니다.

In [ ]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

#### 카프카(Kafka) 설정 
여기에서 Kafka를 설정합니다. 구성되면 설정이 새로 생성된 모든 데이터 로더와 공유되며 각 로더에 대해 Kafka를 설정할 필요가 없습니다. 자세한 설정 은 공식 [문서](https://docs.tigergraph.com/pytigergraph/current/gds/gds#_configurekafka) 를 참조하십시오.

In [ ]:
conn.gds.configureKafka(
    kafka_address="127.0.0.1:9092",
    kafka_security_protocol="SASL_PLAINTEXT",
    kafka_sasl_mechanism="PLAIN",
    kafka_sasl_plain_username="your username",
    kafka_sasl_plain_password="your password"
)

#### 정점 배치 가져오기

In [10]:
%%time
vertex_loader = conn.gds.vertexLoader(
    num_batches=20, 
    attributes=["id", "x", "y"],
    shuffle=True,
    filter_by=None,
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 37.1 ms, sys: 11.9 ms, total: 48.9 ms
Wall time: 25.8 s


In [11]:
%%time
for i, batch in enumerate(vertex_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))

----Batch 0: Shape (157, 4)----
   vid    id                                                  x  y
0    9  1341  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  0
----Batch 1: Shape (121, 4)----
       vid    id                                                  x  y
0  7340080  2332  0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 ...  0
----Batch 2: Shape (131, 4)----
        vid   id                                                  x  y
0  11534338  562  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  2
----Batch 3: Shape (143, 4)----
        vid  id                                                  x  y
0  19922966  66  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 ...  4
----Batch 4: Shape (149, 4)----
       vid    id                                                  x  y
0  1048585  1443  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  4
----Batch 5: Shape (125, 4)----
       vid    id                                                  x  y
0  8388625  2477  0 0 0 0 0 0 0 0 0 0